In [ ]:
metadata = {
    'protocolName': 'OER',
    'author': 'Jehad Abed <jehad.abed@mail.utoronto.com>',
    'description': 'Simple protocol to get started using OT2',
    'apiLevel': '2.9'
    }

### Initialization

In [ ]:
import opentrons.execute
from opentrons import types
import time, json
import pandas as pd
from datetime import datetime, timedelta
import math
import os
import shutil
protocol = opentrons.execute.get_protocol_api('2.9')

In [ ]:
# labware
tiprack_20 = protocol.load_labware('opentrons_96_tiprack_20ul', '7')
tiprack_1000 = protocol.load_labware('opentrons_96_tiprack_1000ul', '6')
left_pipette = protocol.load_instrument('p20_single_gen2', 'left', tip_racks=[tiprack_20]) # pipettes
right_pipette = protocol.load_instrument('p1000_single_gen2', 'right', tip_racks=[tiprack_1000]) # pipettes

# Define custom-made rack of 24 vials , each with capacity 6mL
LABWARE_DEF = json.load(open("Labware/a3md_24_vials_6ml.json"))
LABWARE_LABEL = LABWARE_DEF.get('metadata', {}).get('displayName', 'rack_24_6mL')
LABWARE_SLOT = '4'
labware = protocol.load_labware_from_definition(LABWARE_DEF, LABWARE_SLOT, LABWARE_LABEL)
well_locs = sum(LABWARE_DEF.get('ordering'),[])
vials_24_6mL=[labware,well_locs]

# Define custom made reactor labware with 3x5 reaction wells
LABWARE_DEF = json.load(open("Labware/a3md_3x5_3ecell_v1.json"))
LABWARE_LABEL = LABWARE_DEF.get('metadata', {}).get('displayName', 'reactor_3x5')
LABWARE_SLOT = '2'
labware = protocol.load_labware_from_definition(LABWARE_DEF, LABWARE_SLOT, LABWARE_LABEL)
well_locs = sum(LABWARE_DEF.get('ordering'),[])
reactor_3x5=[labware,well_locs]


# Define vial for electrolyte/waste
LABWARE_DEF = json.load(open("Labware/a3md_100ml_vial_2.json"))
LABWARE_LABEL = LABWARE_DEF.get('metadata', {}).get('displayName', 'vial_100ml')
LABWARE_SLOT = '5'
labware = protocol.load_labware_from_definition(LABWARE_DEF, LABWARE_SLOT, LABWARE_LABEL)
electrolyte_vial = labware['A1']
waste_vial = labware['A2']

# Define vial for washing
LABWARE_DEF = json.load(open("Labware/a3md_vial_240ml.json"))
LABWARE_LABEL = LABWARE_DEF.get('metadata', {}).get('displayName', 'vial_240ml')
LABWARE_SLOT = '3'
vial_240ml= protocol.load_labware_from_definition(LABWARE_DEF, LABWARE_SLOT, LABWARE_LABEL)

# Load Echem holder
Loading_pos={
        'x': 12.13,
        'y': 9.0,
        'z': 100
    }

In [ ]:
#Setting global Opentron parameters

RATE = 0.50 #speed rate

def set_speeds(rate):
    """
    Args:
        rate: reduction factor to the default speed of moving
    Outcome:
        Reduces the speed by which opentron moves
    """
    protocol.max_speeds.update({'X': (600 * rate),'Y': (400 * rate),'Z': (125 * rate),'A': (125 * rate),})
    speed_max = max(protocol.max_speeds.values())
    for instr in protocol.loaded_instruments.values():
        instr.default_speed = speed_max

## Experimental offset value
x_offset = 1
y_offset = -1.2

### Calibration

In [ ]:
CALIBRATION_CROSS_COORDS = {
    '1': {
        'x': 12.13,
        'y': 9.0,
        'z': 0.0
    },
    '3': {
        'x': 380.87,
        'y': 9.0,
        'z': 0.0
    },
    '7': {
        'x': 12.13,
        'y': 258.0,
        'z': 0.0
    }
}
CALIBRATION_CROSS_SLOTS = ['1']

In [ ]:
def calibrate(pipette):
    protocol.home()
    for slot in CALIBRATION_CROSS_SLOTS:
        coordinate = CALIBRATION_CROSS_COORDS[slot]
        location = types.Location(point=types.Point(**coordinate),labware=None)
        pipette.pick_up_tip()
        pipette.move_to(location)
        protocol.pause(f"Confirm left pipette is at slot {slot} calibration cross")
        a = input("can you confirm if the tip is indeed at the center? (y/n)")
        print("You said {}".format(a))
        protocol.resume()
        pipette.return_tip()

In [ ]:
calibrate(left_pipette)
protocol.home()

### Load/Unload Echem Holder

In [ ]:
def load_holder(coordinate):
    """
    Args:
        loadpos: the loading position for the holder
    Output:
        Go to loading position
    """
    
     # Set the rate for how fast things should move
    set_speeds(RATE)
    
    location = types.Location(point=types.Point(**coordinate),labware=None)
    protocol.pause(f"Clear loading area")
    a = input("Make sure loading/unloading position is clear (y/n)")
    protocol.resume()
    right_pipette.move_to(location)
    protocol.pause(f"Confirm holder is loaded")
    a = input("Confirm if the echem holder is correctly loaded? (y/n)")
    protocol.resume()
    location = types.Location(point=types.Point(coordinate['x'],coordinate['y'],coordinate['z']+80),labware=None)
    right_pipette.move_to(location)
    
def unload_holder(coordinate):
    """
    Args:
        coordinate: the loading position for the holder
    Output:
        Go to unloading position
    """
    
     # Set the rate for how fast things should move
    set_speeds(RATE*0.75)
    
    location = types.Location(point=types.Point(coordinate['x'],coordinate['y'],coordinate['z']+80),labware=None)
    right_pipette.move_to(location)
    protocol.pause(f"Clear loading area")
    a = input("Make sure loading/unloading position is clear (y/n)")
    protocol.resume()
    location = types.Location(point=types.Point(**coordinate),labware=None)
    right_pipette.move_to(location)
    protocol.pause(f"Confirm holder is unloaded")
    a = input("Confirm if the echem holder is correctly unloaded? (y/n)")
    protocol.resume()

### Park holder

In [ ]:
def park_holder(labware, parking_pos="C5"):
    """
    Args:
         labware: choose labware location for experiment
         park_pos: parking position in the labware 
    Output:
        Go to loading position
    """
    
     # Set the rate for how fast things should move
    set_speeds(RATE)
    
    current_target_well = parking_pos
    pos = list(labware[0][current_target_well].bottom())[0]
    right_pipette.move_to(types.Location(point=types.Point(pos[0]+x_offset-15, pos[1]-y_offset, pos[2]+70),labware=None))
    protocol.pause(f"Parked...")
    do_alignment = input("Exit parking n/y?")
    protocol.resume()
    right_pipette.move_to(types.Location(point=types.Point(pos[0]+x_offset-15, pos[1]-y_offset, pos[2]+100),labware=None))

### Fill/Empty Electrolyte

In [ ]:
def fill_electrolyte(filename, labware, electrolyte, x_offset=x_offset, y_offset=y_offset):
    """
    Args:
        filename: name of the excel file from which OER experiments
        labware: choose labware location for experiment
        electrolyte: choose source of electrolyte
    Output:
        fills in electrolyte for all wells in labware
    """
    
    
    df = pd.read_csv(filename)
    # Set the rate for how fast things should move
    set_speeds(RATE*1.5)
    right_pipette.pick_up_tip()
    
    
    # level of liquid
    protocol.pause(f"Liquid level in electrolyte")
    a = input("What is the level of liquid in vial in mL?")
    protocol.resume()
    

    z_level=100-int(a)
    
    #fill wells in reactor
    for idx,i in enumerate(df.index.values):
        
        current_target_well = df.loc[i,"Target_well"]
        
        #If volume>1000 ul repeat process until reaching the target volume
        if df.loc[i,"Input_volume"]>1000:
            repeat_fill=int(math.ceil(df.loc[i,"Input_volume"]/1000))
        else:
            repeat_fill=1
        
        current_volume=0
        while repeat_fill>0:
            
            if (df.loc[i,"Input_volume"]-current_volume)>=1000:
                input_volume=1000
            else:
                input_volume=df.loc[i,"Input_volume"]-current_volume
            
            current_volume+=input_volume
            
            right_pipette.flow_rate.aspirate = 2000
            right_pipette.aspirate(input_volume, electrolyte.top(z=-42+-0.62*z_level)) #cm/mL
            z_level+=(input_volume/1000)
            
            pos = list(labware[0][current_target_well].bottom())[0]
            right_pipette.move_to(types.Location(point=types.Point(pos[0]+x_offset, pos[1]-y_offset, pos[2]),labware=None))
            
            right_pipette.flow_rate.dispense = 2000
            right_pipette.dispense(input_volume)
            print(repeat_fill, current_target_well, current_volume)
            repeat_fill-=1
        
        time.sleep(1)
        
    right_pipette.return_tip()

In [ ]:
def empty_electrolyte(labware, disposal, rep=5, skip_list=["B1","B5", "C5"],x_offset=x_offset, y_offset=y_offset):
    """
    Args:
        labware: choose target labware
        disposal: choose disposal location
    Output:
        Empty electrolyte in labware
    """
    
    set_speeds(RATE*1.5)
    right_pipette.pick_up_tip()
    
    
    #fill wells in reactor
    for well_loc in labware[1]:
        if well_loc not in skip_list:
            i=0
            while i<5:
                pos = list(labware[0][well_loc].bottom())[0]
                right_pipette.flow_rate.aspirate = 2000
                right_pipette.aspirate(1000, types.Location(point=types.Point(pos[0]+x_offset, pos[1]-y_offset, pos[2]),labware=None))
                right_pipette.flow_rate.dispense = 2000
                right_pipette.dispense(1000, disposal.top())
                i+=1
        
    right_pipette.return_tip()

In [ ]:
#protocol.home()
#right_pipette.return_tip()
fill_electrolyte("experiments/OER_input.csv", reactor_3x5, electrolyte_vial)

### Washing

In [ ]:
def wash_electrode(labware, well_loc="A1", cycles=1, hold_time=1):
    """
    Args:
        labware: choose target labware
        cycles: number of washing cycles
        hold_time: time per each wash
    Output:
        Empty electrolyte in labware
    """
    
    set_speeds(RATE*1.5)  
    pos = list(labware[well_loc].bottom())[0]
    
    t1=datetime.now()
    right_pipette.move_to(types.Location(point=types.Point(pos[0]+x_offset-15, pos[1]-y_offset, pos[2]+150),labware=None))
                           
    # fill wells in reactor
    i=0
    while i<cycles:
        pos = list(labware[well_loc].bottom())[0]
        
        #dip
        right_pipette.move_to(types.Location(point=types.Point(pos[0]+x_offset-15, pos[1]-y_offset, pos[2]+80),labware=None))
        
        #hold
        time.sleep(hold_time)
        
        #up
        right_pipette.move_to(types.Location(point=types.Point(pos[0]+x_offset-15, pos[1]-y_offset, pos[2]+90),labware=None))
        i+=1
        
    t2=datetime.now()
    deltat=t2-t1
    
    return deltat

In [ ]:
#Test
washing_time = wash_electrode(vial_240ml, hold_time=0, cycles=1)
print(washing_time)

unload_holder(Loading_pos)
#protocol.home()

### Dropcasting

In [ ]:
# protocol run function

def dropcast(filename, target_labware, input_labware, logbook=True, x_offset=x_offset, y_offset=y_offset):
    """
    Args:
        filename: name of the excel file from which dropcasting instructions are to be read
    Output:
        Executes dropcasting in a loop
    """
    
    df = pd.read_csv(filename)
    # Set the rate for how fast things should move
    set_speeds(RATE)
    left_pipette.pick_up_tip()
    
    # commands
    previous_input_well = -1
    
    for idx,i in enumerate(df.index.values):
        
        current_input_well, current_target_well = df.loc[i, "Input_well"], df.loc[i,"Target_well"]
        
        if previous_input_well != current_input_well and previous_input_well!=-1: 
            left_pipette.return_tip()
            left_pipette.pick_up_tip() 

        #pos = list(input_labware[0][current_input_well].bottom())[0]
        #left_pipette.move_to(types.Location(point=types.Point(pos[0]+1.5, pos[1], pos[2]),labware=None))
        #left_pipette.aspirate(df.loc[i,"Input_volume"])
        left_pipette.aspirate(df.loc[i,"Input_volume"], input_labware[0][current_input_well].bottom(z=-4))
        
        
        pos = list(target_labware[0][current_target_well].bottom())[0]
        left_pipette.move_to(types.Location(point=types.Point(pos[0]+x_offset, pos[1]+y_offset, pos[2]-0.5),labware=None))
        left_pipette.dispense(df.loc[i,"Input_volume"])
        left_pipette.blow_out()
        
        #write to logbook
        if logbook:
            df_w = pd.DataFrame({'Timestamp': [datetime.now()],'Experiment_ID': [df.loc[i, "Experiment_ID"]],'User': [metadata['author']],'Target_well': [df.loc[i, "Target_well"]],'Input_well': [df.loc[i, "Input_well"]],'Input_volume': [df.loc[i, "Input_volume"]]})
            df_w.to_csv(filename[0:12]+'dropcasting_logbook.csv', mode='a', header=False)
        
        previous_input_well = df.loc[i,"Input_well"]
        
    left_pipette.return_tip()

In [ ]:
## Debugging/reset
#left_pipette .drop_tip()
#left_pipette.return_tip()
#protocol.home()
dropcast("experiments/dropcasting_input.csv", reactor_3x5, vials_24_6mL)

### Run Echem

In [ ]:
# protocol run function

def run_OER_old(filename, labware, wait_time=0, calibrate_well="B1", logbook=True):
    """
    Args:
        filename: name of the excel file from which OER experiments
        wait_time: stay hold in place for potentiostat in seconds
        time_shift: time moving between cells
        #LSV: 80 sec for 10 mV/s + 2 sec OCV
        #Full: 1780 sec
        #Movement between cells: 2 sec
    Output:
        Executes dropcasting in a loop
    """
    
    df = pd.read_csv(filename)
    # Set the rate for how fast things should move
    set_speeds(RATE*0.75)
    
    # load holder
    load_holder(Loading_pos)
    
    # electrodes alignment check
    protocol.pause(f"Electrode alignment check")
    do_alignment = input("Do you want to check alignment n/y?")
    protocol.resume()
    
    while do_alignment=="y":
        current_target_well = calibrate_well
        pos = list(labware[0][current_target_well].bottom())[0]
        right_pipette.move_to(types.Location(point=types.Point(pos[0]+x_offset-15, pos[1]-y_offset, pos[2]+100),labware=None))
        protocol.pause(f"Align electrodes")
        msg = input("To proceed press <Enter>")
        protocol.resume()
        right_pipette.move_to(types.Location(point=types.Point(pos[0]+x_offset-15, pos[1]-y_offset, pos[2]+71),labware=None))
        protocol.pause(f"Repeat alignment")
        resume = input("Do you want to repeat alignment procedure n/y?")
        protocol.resume()
        right_pipette.move_to(types.Location(point=types.Point(pos[0]+x_offset-15, pos[1]-y_offset, pos[2]+100),labware=None))
        if resume=="n":
            break 
        unload_holder(Loading_pos)

    
    time1 = datetime.now()
    
    for idx,i in enumerate(df.index.values):
        
        
        #go to wells in reactor
        current_target_well = df.loc[i,"Target_well"]
        pos = list(labware[0][current_target_well].bottom())[0]
        right_pipette.move_to(types.Location(point=types.Point(pos[0]+x_offset-15, pos[1]-y_offset, pos[2]+71),labware=None))
        
        
        #counter to synchronise and start the potentiostat
        if i==0:
            protocol.pause(f"Turn potentiostat")
            print("Turn potentiostat in 5")
            counter=5
            while counter>0:
                print(f'{counter}...')
                time.sleep(1)
                counter-=1
            print('Start!')
            protocol.resume()
            
        #Delete run file
        if os.path.exists("run"):
            os.remove("run")
            
       # wait until trig_pot.exe triggers the potentiostat for next measurement
        while os.path.exists("run")==False:
            pass
        
        time.sleep(wait_time)
        
        #write to logbook
        if logbook:
            df_w = pd.DataFrame({'Timestamp': [datetime.now()],'Experiment_ID': [df.loc[i, "Experiment_ID"]],'User': [metadata['author']],'Target_well': [df.loc[i, "Target_well"]]})
            df_w.to_csv(filename[0:12]+'OER_logbook.csv', mode='a', header=False)
        
        #go to washing procedure
        wash_electrode(vial_240ml, hold_time=0, cycles=1)
        t2 = datetime.now()
        
        #move back from washing position to next well
        if i<len(df.index.values)-1:
            current_target_well = df.loc[i+1,"Target_well"]
            pos = list(labware[0][current_target_well].bottom())[0]
            right_pipette.move_to(types.Location(point=types.Point(pos[0]+x_offset-15, pos[1]-y_offset, pos[2]+155),labware=None))
        elif i==len(df.index.values)-1:
            current_target_well = df.loc[i,"Target_well"]
            pos = list(labware[0][current_target_well].bottom())[0]
            right_pipette.move_to(types.Location(point=types.Point(pos[0]+x_offset-15, pos[1]-y_offset, pos[2]+155),labware=None))

        #shift time after first test only
        if i!=0:
            time_shift=t2-t1
            print(i, time_shift)
                    
        t1 = datetime.now()
        
   #Parking the electrode
    park_holder(reactor_3x5)
            
    #unload holder
    unload_holder(Loading_pos)
        

In [ ]:
# protocol run function

def run_OER(filename, labware, wait_time=0, calibrate_well="B1", logbook=True):
    """
    Args:
        filename: name of the excel file from which OER experiments
        wait_time: stay hold in place for potentiostat in seconds
        time_shift: time moving between cells
        #LSV: 80 sec for 10 mV/s + 2 sec OCV
        #Full: 1780 sec
        #Movement between cells: 2 sec
    Output:
        Executes dropcasting in a loop
    """
    
    df = pd.read_csv(filename)
    # Set the rate for how fast things should move
    set_speeds(RATE*0.75)
    
    # load holder
    load_holder(Loading_pos)
    
    # electrodes alignment check
    protocol.pause(f"Electrode alignment check")
    do_alignment = input("Do you want to check alignment n/y?")
    protocol.resume()
    
    while do_alignment=="y":
        current_target_well = calibrate_well
        pos = list(labware[0][current_target_well].bottom())[0]
        right_pipette.move_to(types.Location(point=types.Point(pos[0]+x_offset-15, pos[1]-y_offset, pos[2]+100),labware=None))
        protocol.pause(f"Align electrodes")
        msg = input("To proceed press <Enter>")
        protocol.resume()
        right_pipette.move_to(types.Location(point=types.Point(pos[0]+x_offset-15, pos[1]-y_offset, pos[2]+71),labware=None))
        protocol.pause(f"Repeat alignment")
        resume = input("Do you want to repeat alignment procedure n/y?")
        protocol.resume()
        right_pipette.move_to(types.Location(point=types.Point(pos[0]+x_offset-15, pos[1]-y_offset, pos[2]+100),labware=None))
        if resume=="n":
            break 
        unload_holder(Loading_pos)

    
    
    #Delete run files before starting
    if os.path.exists("end"):
        os.remove("end")
        
    if os.path.exists("trigger"):
        os.remove("trigger")
    
    for idx,i in enumerate(df.index.values):
        
        t1 = datetime.now()
        
        #writing previous completed entry to logbook
        if logbook and i>0:
            df_w = pd.DataFrame({'Timestamp': [datetime.now()+timedelta(hours=-4)],'Experiment_ID': [df.loc[i-1, "Experiment_ID"]],'User': [metadata['author']],'Target_well': [df.loc[i-1, "Target_well"]]})
            df_w.to_csv(filename[0:12]+'OER_logbook.csv', mode='a', header=False)
        
        #go to washing procedure
        wash_electrode(vial_240ml, hold_time=0, cycles=1)
        
        #go to well pos
        current_target_well = df.loc[i,"Target_well"]
        pos = list(labware[0][current_target_well].bottom())[0]
        right_pipette.move_to(types.Location(point=types.Point(pos[0]+x_offset-15, pos[1]-y_offset, pos[2]+155),labware=None))
        
        
        #counter to synchronise and start the potentiostat
        if i==0:
            protocol.pause(f"Turn potentiostat")
            print("Turn potentiostat in 5")
            counter=5
            while counter>0:
                print(f'{counter}...')
                time.sleep(1)
                counter-=1
            print('Start!')
            protocol.resume()
            
        
        right_pipette.move_to(types.Location(point=types.Point(pos[0]+x_offset-15, pos[1]-y_offset, pos[2]+71),labware=None))
        
        t2 = datetime.now()
        
        #moving between washing and well time
        time_shift=t2-t1
        print(i, time_shift.total_seconds())
        
        #remove previous end file
        if os.path.exists("end"):
            os.remove("end")
        
        #create and scp trigger to trigger potentiostat
        path = '/var/lib/jupyter/notebooks/A3MD/'
        shutil.copy2(path+'experiments/trigger', path)
        
       
        # wait until trig_pot.exe triggers the potentiostat for next measurement
        while os.path.exists("end")==False:
                pass
        
        #add extra wait time if needed
        time.sleep(wait_time)
        
        #remove trigger
        if os.path.exists('trigger'):
            os.remove('trigger')
        

    #write last entry in logbook
    if logbook:
        i = len(df.index.values)-1
        df_w = pd.DataFrame({'Timestamp': [datetime.now()+timedelta(hours=-4)],'Experiment_ID': [df.loc[i, "Experiment_ID"]],'User': [metadata['author']],'Target_well': [df.loc[i, "Target_well"]]})
        df_w.to_csv(filename[0:12]+'OER_logbook.csv', mode='a', header=False)
        
        
   #Parking the electrode
    park_holder(reactor_3x5)
            
    #unload holder
    unload_holder(Loading_pos)
        

In [ ]:
#protocol.home()
#fill_electrolyte("experiments/OER_input.csv",reactor_3x5, electrolyte_vial)
#load_holder(Loading_pos)
#park_holder(reactor_3x5)
run_OER("experiments/OER_input.csv", reactor_3x5)